In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import pandas as pd
import numpy as np
import keras


In [3]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
concrete_data.shape

(1030, 9)

In [8]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [9]:
concrete_data.isnull().sum()

,0
Cement,0
Blast Furnace Slag,0
Fly Ash,0
Water,0
Superplasticizer,0
Coarse Aggregate,0
Fine Aggregate,0
Age,0
Strength,0


In [10]:
concrete_data_columns = concrete_data.columns

In [11]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [12]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [13]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [14]:
n_cols = predictors_norm.shape[1] # number of predictors

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Input

In [17]:
def regression_model():

  model = Sequential()
  model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer = 'adam' , loss = 'mean_squared_error')
  return model


In [19]:
model = regression_model()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 5s - 232ms/step - loss: 1645.7141 - val_loss: 1138.1423
Epoch 2/100
23/23 - 0s - 5ms/step - loss: 1504.7804 - val_loss: 1015.4066
Epoch 3/100
23/23 - 0s - 5ms/step - loss: 1286.4355 - val_loss: 830.5183
Epoch 4/100
23/23 - 0s - 5ms/step - loss: 967.5862 - val_loss: 598.7471
Epoch 5/100
23/23 - 0s - 5ms/step - loss: 611.7303 - val_loss: 378.5710
Epoch 6/100
23/23 - 0s - 5ms/step - loss: 356.8783 - val_loss: 240.6076
Epoch 7/100
23/23 - 0s - 5ms/step - loss: 262.7725 - val_loss: 189.6993
Epoch 8/100
23/23 - 0s - 5ms/step - loss: 237.1312 - val_loss: 177.0239
Epoch 9/100
23/23 - 0s - 5ms/step - loss: 223.0654 - val_loss: 170.6606
Epoch 10/100
23/23 - 0s - 5ms/step - loss: 211.9755 - val_loss: 166.7890
Epoch 11/100
23/23 - 0s - 5ms/step - loss: 203.5241 - val_loss: 165.9115
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 196.4019 - val_loss: 162.6279
Epoch 13/100
23/23 - 0s - 5ms/step - loss: 190.6699 - val_loss: 158.6075
Epoch 14/100
23/23 - 0s - 5ms/step - loss: 185.3388 -

In [21]:
def regression_model_2():

  model = Sequential()
  model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer = 'adam' , loss = 'mean_squared_error')
  return model

model_2 = regression_model_2()
model_2.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 4s - 163ms/step - loss: 1564.2361 - val_loss: 902.2222
Epoch 2/100
23/23 - 0s - 6ms/step - loss: 716.0077 - val_loss: 374.6545
Epoch 3/100
23/23 - 0s - 6ms/step - loss: 293.5840 - val_loss: 204.9769
Epoch 4/100
23/23 - 0s - 6ms/step - loss: 232.0785 - val_loss: 194.5786
Epoch 5/100
23/23 - 0s - 6ms/step - loss: 207.3256 - val_loss: 214.2731
Epoch 6/100
23/23 - 0s - 6ms/step - loss: 186.5420 - val_loss: 201.0257
Epoch 7/100
23/23 - 0s - 6ms/step - loss: 169.5857 - val_loss: 202.6845
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 151.1514 - val_loss: 195.8694
Epoch 9/100
23/23 - 0s - 6ms/step - loss: 137.1292 - val_loss: 196.0245
Epoch 10/100
23/23 - 0s - 6ms/step - loss: 120.3134 - val_loss: 210.1055
Epoch 11/100
23/23 - 0s - 6ms/step - loss: 104.8459 - val_loss: 201.7681
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 92.9174 - val_loss: 182.8038
Epoch 13/100
23/23 - 0s - 5ms/step - loss: 83.1078 - val_loss: 220.9075
Epoch 14/100
23/23 - 0s - 6ms/step - loss: 74.8671 - val_loss: 176.6228